In [1]:
import os
os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'gen'))
# sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models', 'eval'))

import torch
import pprint
import json
from importlib import import_module, reload
from gen.utils.image_util import decompress_mask as util_decompress_mask
import gen.constants as constants

In [2]:
import json
import numpy as np
from PIL import Image
from datetime import datetime
from eval.eval import Eval
from env.thor_env import ThorEnv
from eval.eval_task import EvalTask
from collections import defaultdict

In [3]:
date = '20200506'

In [4]:
import logging
split_name = 'valid_unseen'
logging.basicConfig(filename='/root/data_alfred/splits/collect_states_{}_{}.log'.format(date, split_name),level=logging.DEBUG)

In [5]:
class args:
    pass

# settings
args.splits = '/root/data_alfred/splits/apr25.json'
args.data = '/root/data_alfred/json_feat_2.1.0'
args.reward_config = 'models/config/rewards.json'
args.eval_split = 'valid_seen'
args.model_path = '/root/data_alfred/pretrained_model/model:seq2seq_im_mask,name:base30_pm010_sg010_01/best_seen.pth'
args.model = 'models.model.seq2seq_im_mask'
args.preprocess = False
args.shuffle = False
args.gpu = False
args.num_threads = 1  

# eval params
args.max_steps = 1000
args.max_fails = 10

# eval settings
args.subgoals = ''
args.smooth_nav = False
args.skip_model_unroll_with_expert = False
args.no_teacher_force_unroll_with_expert = False

# debug
args.debug = False
args.fast_epoch = False

In [6]:
def load_task_json(task):
    '''
    load preprocessed json from disk
    '''
    json_path = os.path.join(args.data, task['task'], '%s' % 'pp', 'ann_%d.json' % task['repeat_idx'])
    with open(json_path) as f:
        data = json.load(f)
    return data

In [7]:
def decompress_mask(compressed_mask):
    '''
    decompress mask from json files
    '''
    mask = np.array(util_decompress_mask(compressed_mask))
    mask = np.expand_dims(mask, axis=0)
    return mask

class CollectStates(EvalTask): 
    
    object_state_list = ['isToggled', 'isBroken', 'isFilledWithLiquid', 'isDirty',
                  'isUsedUp', 'isCooked', 'ObjectTemperature', 'isSliced',
                  'isOpen', 'isPickedUp', 'mass', 'receptacleObjectIds']

    object_symbol_list = constants.OBJECTS
    
    @classmethod
    def get_object_list(cls, traj_data):
        # TODO rewrite
        object_list = [ob['objectName'] for ob in traj_data['scene']['object_poses']]
        for ob in object_list:
            assert ob.split('_')[0] in constants.OBJECTS
        return object_list
    
    @classmethod
    def get_object_states(cls, metadata):
        object_states = defaultdict(dict)
        for ob in metadata['objects']:
            symbol = ob['name'].split('_')[0]
            # assert symbol in cls.object_symbol_list
            object_states[ob['name']]['symbol'] = symbol
            object_states[ob['name']]['states'] = {state:ob[state] for state in cls.object_state_list}
            object_states[ob['name']]['states']['parentReceptacles'] = ob['parentReceptacles'][0].split('|')[0] if ob['parentReceptacles'] is not None else None
        return object_states

    @classmethod    
    def divide_objects_by_change(cls, object_states_curr, object_states_last):
        objects_unchanged = []
        objects_changed = []
        for ob_name in object_states_last.keys():
            changed = False
            for state in cls.object_state_list + ['parentReceptacles']:
                if state in object_states_last[ob_name]['states'].keys():
                    if object_states_last[ob_name]['states'][state] != object_states_curr[ob_name]['states'][state]:
                        changed = True
            if changed == False:
                objects_unchanged.append(ob_name)
            else:
                objects_changed.append(ob_name)
        return objects_changed, objects_unchanged

    @classmethod  
    def get_unchanged_symbols(cls, objects_changed, objects_unchanged, symbol_set):
        objects_symbols_changed = [ob_name.split('_')[0] for ob_name in objects_changed]
        objects_symbols_unchanged = [ob_name.split('_')[0] for ob_name in objects_unchanged]
        return list((set(objects_symbols_unchanged) - set(objects_symbols_changed)) & symbol_set)

    @classmethod
    def get_object_symbols_present_in_scene(cls, traj_data):
        object_list = [ob['objectName'] for ob in traj_data['scene']['object_poses']]
        extracted_symbols = [ob.split('_')[0] for ob in object_list]
        # for symbol in extracted_symbols:
        #     assert symbol in cls.object_symbol_list
        return extracted_symbols

    @classmethod
    def get_receptacle_symbols_present_in_scene(cls, metadata):
        receptacle_list = [ob['name'] for ob in metadata['objects'] if ob['receptacle']]
        extracted_symbols = [ob.split('_')[0] for ob in receptacle_list]
        return extracted_symbols

    @classmethod
    def get_visibility(cls, metadata, object_symbols, receptacle_symbols):
        visible_objects = {ob:False for ob in object_symbols}
        visible_receptacles = {recp:False for recp in receptacle_symbols}
        for ob in metadata['objects']:
            if ob['visible']:
                symbol = ob['name'].split('_')[0]
                if ob['receptacle']:
                    visible_receptacles[symbol] = True
                else:
                    visible_objects[symbol] = True
        return [ob for ob in visible_objects.keys() if visible_objects[ob]], [recp for recp in visible_receptacles.keys() if visible_receptacles[recp]]
        
    @classmethod
    # add lock back
    def evaluate(cls, env, r_idx, traj_data, args, successes, failures, results, ct):
        
        logging.info('count = {}'.format(ct))
        logging.info(traj_data['root'])
        logging.info(traj_data['task_type'])
        
        # setup scene
        reward_type = 'dense'
        cls.setup_scene(env, traj_data, r_idx, args, reward_type=reward_type)
        
        # goal instr
        goal_instr = traj_data['turk_annotations']['anns'][r_idx]['task_desc']
        
        # ground-truth actions
        # e.g. ['LookDown_15', 'MoveAhead_25', 'MoveAhead_25', ... '<<stop>>']
        groundtruth_action_low = [a['discrete_action']['action'] for a in traj_data['plan']['low_actions']]
        groundtruth_action_low.append(cls.STOP_TOKEN)
        # e.g. [0,0,0, ... , 11]
        groundtruth_subgoal_alignment = traj_data['num']['low_to_high_idx']
        
        # ground-truth mask and valid interacts
        # lenght = total T
        # e.g. [0,1,0, ... , 1]
        action_low_group = [a for a_group in traj_data['num']['action_low'] for a in a_group]
        groundtruth_valid_interacts = [a['valid_interact'] for a in action_low_group]
        # len=num timestep with valid interact, np shape (1 , 300 , 300)
        groundtruth_low_mask = [decompress_mask(a['mask']) for a in action_low_group if a['mask'] is not None]
        
        # ground-truth high-level subgoals
        # e.g. ['GotoLocation', 'PickupObject', 'SliceObject', 'GotoLocation', 'PutObject', ... 'NoOp']
        groundtruth_action_high = [a['discrete_action']['action'] for a in traj_data['plan']['high_pddl']]
        
        assert len(groundtruth_action_low) == len(groundtruth_subgoal_alignment) == len(groundtruth_valid_interacts)
        assert len(groundtruth_action_high) == groundtruth_subgoal_alignment[-1] + 1
        assert sum(groundtruth_valid_interacts) == len(groundtruth_low_mask)

        # initialize state dictionary for all timesteps
        states = []

        # get symbols and initial object states
        event = env.last_event
        obj_symbol_set = set(cls.get_object_symbols_present_in_scene(traj_data))
        receptacle_symbol_set = set(cls.get_receptacle_symbols_present_in_scene(event.metadata))
        object_states_last = cls.get_object_states(event.metadata) # includes receptacles
        
        done, success = False, False
        fails = 0
        t = 0
        reward = 0
        action, mask = None, None
        interact_ct = 0
        high_idx = -1
        while not done:            
            # if last action was stop, break
            if action == cls.STOP_TOKEN:
                done = True
                logging.info("predicted STOP")
                break
            
            # transition to next subgoal
            if high_idx < groundtruth_subgoal_alignment[t]:
                high_idx = groundtruth_subgoal_alignment[t]
#                 event = env.last_event
                object_states_curr = cls.get_object_states(event.metadata)
                visible_objects, visible_receptacles = cls.get_visibility(event.metadata, obj_symbol_set, receptacle_symbol_set)
                objects_changed, objects_unchanged = cls.divide_objects_by_change(object_states_curr, object_states_last)
                states.append({
                    'time_step': t,
                    'raw_object_metadata': event.metadata['objects'],
                    'receptacle_states_delta': cls.get_unchanged_symbols(objects_changed, objects_unchanged, receptacle_symbol_set),
                    'object_states_delta': cls.get_unchanged_symbols(objects_changed, objects_unchanged, obj_symbol_set),
                    'visible_objects': visible_objects,
                    'visible_receptacles': visible_receptacles,
                    'subgoals': groundtruth_action_high[high_idx]
                })
                object_states_last = object_states_curr
            
            # collect groundtruth action and mask
            # single string
            action = groundtruth_action_low[t]
            # expect (300, 300)
            if groundtruth_valid_interacts[t]:
                mask = groundtruth_low_mask[interact_ct][0]
                interact_ct += 1
            else:
                mask = None

            # interact with the env
            t_success, event, _, err, _ = env.va_interact(action, interact_mask=mask, smooth_nav=args.smooth_nav, debug=args.debug)
            if not t_success:
                fails += 1
                if fails >= args.max_fails:
                    logging.info("Interact API failed %d times" % fails + "; latest error '%s'" % err)
                    break            
 
            # next time-step
            t_reward, t_done = env.get_transition_reward()
            reward += t_reward
            t += 1
        
        # make sure we have used all masks
        assert interact_ct == sum(groundtruth_valid_interacts)
        
        # check if goal was satisfied
        goal_satisfied = env.get_goal_satisfied()
        if goal_satisfied:
            print("Goal Reached")
            success = True
        assert success

        # goal_conditions
        pcs = env.get_goal_conditions_met()
        goal_condition_success_rate = pcs[0] / float(pcs[1])

        # SPL
        path_len_weight = len(traj_data['plan']['low_actions'])
        s_spl = (1 if goal_satisfied else 0) * min(1., path_len_weight / float(t))
        pc_spl = goal_condition_success_rate * min(1., path_len_weight / float(t))

        # path length weighted SPL
        plw_s_spl = s_spl * path_len_weight
        plw_pc_spl = pc_spl * path_len_weight
        
        
        log_entry = {'trial': traj_data['task_id'],
                     'type': traj_data['task_type'],
                     'repeat_idx': int(r_idx),
                     'goal_instr': goal_instr,
                     'completed_goal_conditions': int(pcs[0]),
                     'total_goal_conditions': int(pcs[1]),
                     'goal_condition_success': float(goal_condition_success_rate),
                     'success_spl': float(s_spl),
                     'path_len_weighted_success_spl': float(plw_s_spl),
                     'goal_condition_spl': float(pc_spl),
                     'path_len_weighted_goal_condition_spl': float(plw_pc_spl),
                     'path_len_weight': int(path_len_weight),
                     'reward': float(reward)}
        if success:
            successes.append(log_entry)
        else:
            failures.append(log_entry)

        # overall results
        results['all'] = cls.get_metrics(successes, failures)

        logging.info("-------------")
        logging.info("SR: %d/%d = %.3f" % (results['all']['success']['num_successes'],
                                    results['all']['success']['num_evals'],
                                    results['all']['success']['success_rate']))
        logging.info("GC: %d/%d = %.3f" % (results['all']['goal_condition_success']['completed_goal_conditions'],
                                    results['all']['goal_condition_success']['total_goal_conditions'],
                                    results['all']['goal_condition_success']['goal_condition_success_rate']))
        logging.info("PLW SR: %.3f" % (results['all']['path_length_weighted_success_rate']))
        logging.info("PLW GC: %.3f" % (results['all']['path_length_weighted_goal_condition_success_rate']))
        logging.info("-------------")

        # task type specific results
        task_types = ['pick_and_place_simple', 'pick_clean_then_place_in_recep', 'pick_heat_then_place_in_recep',
                      'pick_cool_then_place_in_recep', 'pick_two_obj_and_place', 'look_at_obj_in_light',
                      'pick_and_place_with_movable_recep']
        for task_type in task_types:
            task_successes = [s for s in (list(successes)) if s['type'] == task_type]
            task_failures = [f for f in (list(failures)) if f['type'] == task_type]
            if len(task_successes) > 0 or len(task_failures) > 0:
                results[task_type] = cls.get_metrics(task_successes, task_failures)
            else:
                results[task_type] = {}

        logging.info("Goal Reached")
        outpath = os.path.join(traj_data['root'], 'pp', 'subgoal_states.json')
        logging.info('outpath = {}'.format(outpath))
        with open(outpath, 'w') as f:
            json.dump(states, f)

        logging.info("----------------------------------------")
 
        return success, states, outpath

In [8]:
with open(args.splits, 'r') as f:
    splits = json.load(f)

In [9]:
splits.keys()

dict_keys(['tests_seen', 'tests_unseen', 'train', 'valid_seen', 'valid_unseen', 'train_sanity', 'train_sanity_v1', 'valid_seen_v1', 'valid_unseen_v1'])

In [ ]:
%%capture capt

split_name = 'valid_unseen'

env = ThorEnv()

success_ct = 0
r_idx = 0
success_outpaths = []
successes = []
failures = []
results = {}
failed_roots = []
tasks = [task for task in splits[split_name] if task['repeat_idx'] == 0]
tot_ct = len(tasks)

ct = 0
for task in tasks:
    if task['repeat_idx'] == 0:
        print(ct, '/', tot_ct)
        traj_data = load_task_json(task)
        print(traj_data['root'])
        try:
            success, states, outpath = CollectStates.evaluate(env, r_idx, traj_data, args, successes, failures, results, ct)
            success_ct += 1
            success_outpaths.append(outpath)
        except:
            logging.info('FAILED: {}'.format(traj_data['root']))
            print('-----FAILED----')
            failed_roots.append(traj_data['root'])
        
        ct += 1

In [11]:
capt.stdout

"ThorEnv started.\n0 / 255\n/root/data_alfred/json_feat_2.1.0/look_at_obj_in_light-CellPhone-None-FloorLamp-219/trial_T20190908_044113_026049\nResetting ThorEnv\nTask: Pick up the cell phone and look at it by the light of the lamp\nhit invalid action.\nError in to_thor_api_exec ACTION= <<stop>> \nGoal Reached\n1 / 255\n/root/data_alfred/json_feat_2.1.0/pick_heat_then_place_in_recep-Apple-None-SinkBasin-10/trial_T20190907_020258_749030\nResetting ThorEnv\nTask: Get an apple from the sink and heat it up in the microwave\nhit invalid action.\nError in to_thor_api_exec ACTION= <<stop>> \nGoal Reached\n2 / 255\n/root/data_alfred/json_feat_2.1.0/pick_clean_then_place_in_recep-Bowl-None-Cabinet-10/trial_T20190909_061130_844814\nResetting ThorEnv\nTask: Put a washed bowl away in a kitchen cabinet.\nhit invalid action.\nError in to_thor_api_exec ACTION= <<stop>> \nGoal Reached\n3 / 255\n/root/data_alfred/json_feat_2.1.0/pick_and_place_with_movable_recep-Spoon-Mug-CounterTop-10/trial_T20190907_1

In [14]:
with open('/root/data_alfred/splits/collect_states_{}_{}_notebook_capt_stdout.log'.format(date, split_name), 'w') as f:
    f.write(capt.stdout)
    
with open('/root/data_alfred/splits/collect_states_{}_{}_notebook_capt_stderr.log'.format(date, split_name), 'w') as f:
    f.write(capt.stderr)

In [ ]:
# Debug one 

env = ThorEnv()
r_idx = splits['train'][2]['repeat_idx']
traj_data_dum = load_task_json(splits['train'][2])
states, outpath = CollectStates.evaluate(env, r_idx, traj_data, args, 0)

ThorEnv started.
Resetting ThorEnv
Task: Place a slice of cold lettuce on the white table.
1 Spoon_1d6d1e51(Clone)_copy_47
1 Spoon_1d6d1e51(Clone)_copy_47
1 Spoon_1d6d1e51(Clone)_copy_47
1 Lettuce_17_Slice_7
1 Lettuce_17_Slice_7
1 Lettuce_17_Slice_7


In [150]:
i = 11

print(states[i]['time_step'])
print()
print(states[i]['subgoals'])
print()
for x in states[i]['object_states_delta']:
    print(x)
print()  
for x in states[i]['receptacle_states_delta']:
    print(x)
print() 
for x in states[i]['visible_objects']:
    print(x)
print() 
for x in states[i]['visible_receptacles']:
    print(x)

67

NoOp

Apple
Egg
Pot
Bread
SoapBottle
Potato
DishSponge
ButterKnife
Tomato
Bowl
SaltShaker
Knife
Mug
Pan
Fork
Plate
Cup
Spatula
Kettle
SprayBottle
Glassbottle
WineBottle
CellPhone
Spoon
PepperShaker

Mug
Microwave
Drawer
CoffeeMachine
Bowl
Pot
Fridge
Plate
Cup
SinkBasin
CounterTop
Pan
Cabinet
GarbageCan
Toaster
StoveBurner

Apple
SoapBottle
Lettuce
ButterKnife
Fork
Kettle
WineBottle

Mug
Fridge
Cup
DiningTable
